# Collecte des données via l'API

In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
# clé de l'API
API_KEY = 'FuGSHa-GIePXqLpg_xrJwhz_DPy7g9sCRm_HAmvUix8ohqLeof8Dy4l4tD3m_pqzsh8tPhjMogzvAwchwxt2Qwr88jh0PRf7O4kwQqzHO8HcNEQyotUad2bM5ONhZHYx'

In [3]:
# Yelp's GraphQL endpoint
url = "https://api.yelp.com/v3/graphql"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/graphql",
}

In [4]:
businesses = pd.DataFrame(columns=["id", "name", "review_count", "rating", "reviews"])

In [5]:
count = 200
limit = 50
for offset in range(0, count, limit):
    query = f'{{\n\
        search(\
            categories: "restaurants", \
            location: "US", \
            offset: {offset}, \
            limit: {limit}\
        ) {{\n\
            business {{\n\
                id\n\
                name\n\
                review_count\n\
                rating\n\
                reviews {{\n\
                    text\n\
                    rating\n\
                }}\n\
            }}\n\
        }}\n\
    }}'
    response = requests.post(url, headers=headers, data=query)
    data = response.json()
    for business in data.get("data", {}).get("search", {}).get("business", []):
        businesses = businesses.append(
            {
                "id": business.get("id"),
                "name": business.get("name"),
                "review_count": business.get("review_count"),
                "rating": business.get("rating"),
                "reviews": business.get("reviews")
            }, ignore_index=True)

In [6]:
businesses.shape

(200, 5)

In [8]:
# query = """{
#   search(categories: "restaurants", location: "US", offset: 50, limit: 50) {
#     business {
#       id
#       name
#       review_count
#       rating
#       reviews(filter: { rating: { in: "[1,2]" } }) {
#         text
#         rating
#       }
#     }
#   }
# }"""

In [9]:
businesses.head()

,id,name,review_count,rating,reviews
0,f-m7-hyFzkf0HSEeQ2s-9A,Fog Harbor Fish House,9276,4.5,[{'text': 'It was a wonderful experience. Oyst...
1,oT08T3Vpn1I7jDmrBBRMTw,House of Prime Rib,8536,4.0,[{'text': 'Haven't been to the House of Prime ...
2,n6L5VIGunR51-D55C-eYeQ,Foreign Cinema,5377,4.0,[{'text': 'We had an amazing meal! We came wit...
3,WavvLdfdP6g8aZTtbBQHTw,Gary Danko,5853,4.5,[{'text': 'I've been coming to Gary Danko sinc...
4,KkAVX3Wb7E3lP5F_f_8pXg,Liholiho Yacht Club,2627,4.5,[{'text': 'AMAZING food. We were served by Cha...


In [10]:
businesses.describe()

,rating
count,200.00000
mean,4.22750
std,0.27362
min,3.50000
25%,4.00000
50%,4.00000
75%,4.50000
max,5.00000


In [11]:
businesses["reviews"]

0      [{'text': 'It was a wonderful experience. Oyst...
1      [{'text': 'Haven't been to the House of Prime ...
2      [{'text': 'We had an amazing meal! We came wit...
3      [{'text': 'I've been coming to Gary Danko sinc...
4      [{'text': 'AMAZING food. We were served by Cha...
                             ...                        
195    [{'text': 'Everything, great food, adorable pl...
196    [{'text': 'Popular and delicious Mexican resta...
197    [{'text': 'I'm the type of person who frequent...
198    [{'text': 'Let me start by saying, this was on...
199    [{'text': 'Not going to lie. If anyone were to...
Name: reviews, Length: 200, dtype: object

In [12]:
businesses.to_csv("business_dataset.csv", sep='\t', encoding='utf-8')